## Importing Modules

In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import matplotlib.pyplot as plt
import seaborn
import numpy
import json 
import numpy as np

C:\Users\User\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Getting the Universities Links

In [2]:
url = 'https://iisma.kemdikbud.go.id/info/host-universities-list/'
regions_name = ['America & Canada','Asia','Australia','Europe','UK & Ireland']
iisma = requests.get(url).text
iisma_bs = BeautifulSoup(iisma,'lxml')
regions = iisma_bs.find_all('div',class_='elementor-image-carousel swiper-wrapper')
dict_ = {}
image_list = []
for i,region in enumerate(regions):
    links = []
    universities = region.find('a')
    images = region.find_all("img")
    for image in images:
        image_list.append(image['src'])
    link = universities['href']
    links = link.split(',')
    dict_[regions_name[i]] = links

In [3]:
dict_

{'America & Canada': ['https://iisma.kemdikbud.go.id/info/31-boston-university-metropolitan-college/',
  'https://iisma.kemdikbud.go.id/info/56-arizona-state-university/',
  'https://iisma.kemdikbud.go.id/info/05-the-university-of-pennsylvania-college-of-liberal-and-professional-studies/',
  'https://iisma.kemdikbud.go.id/info/06-yale-university/',
  'https://iisma.kemdikbud.go.id/info/29-penn-state-university',
  'https://iisma.kemdikbud.go.id/info/30-university-of-california-davis/',
  'https://iisma.kemdikbud.go.id/info/35-uc-chile/',
  'https://iisma.kemdikbud.go.id/info/45-michigan-state-university/',
  'https://iisma.kemdikbud.go.id/info/64-university-of-colorado-boulder/',
  'https://iisma.kemdikbud.go.id/info/13-university-of-british-columbia/',
  'https://iisma.kemdikbud.go.id/info/42-university-of-waterloo/',
  'https://iisma.kemdikbud.go.id/info/46-western-university/',
  'https://iisma.kemdikbud.go.id/info/03-university-of-chicago/',
  'https://iisma.kemdikbud.go.id/info/21

### Getting the Location for each University

In [70]:
university_list = []
location_list = []
uni_data_dict = {}
for region in dict_:
    for university in dict_[region]:
        university_page = requests.get(university).text
        university_bs = BeautifulSoup(university_page,'lxml')
        university_name = university_bs.find("h4").text
        try:
            location = university_bs.find('div',class_="elementor-column elementor-col-50 elementor-top-column elementor-element elementor-element-c31e763")
            location = location.find('div',class_="elementor-element elementor-element-a5a8329 elementor-widget elementor-widget-text-editor")
        except:
            try:
                location = university_bs.find('div',class_='elementor-column elementor-col-50 elementor-top-column elementor-element elementor-element-eae871c')
                location = location.find('div',class_='elementor-element elementor-element-99457f3 elementor-widget elementor-widget-text-editor')
            except:
                location = university_bs.find('div',class_='elementor-column elementor-col-100 elementor-inner-column elementor-element elementor-element-b4f5966')
                location = location.find('div',class_='elementor-element elementor-element-a5a8329 elementor-widget elementor-widget-text-editor')
        location = location.find('div',class_='elementor-widget-container')
        location = location.p.text
        university_list.append(university_name)
        location_list.append(location)

In [71]:
region_list = []
for region in dict_.keys():
    for i in dict_[region]:
        region_list.append(region)

In [72]:
uni_data_dict["University"] = university_list
uni_data_dict["Location"] = location_list
uni_data_dict["Images"] = image_list
uni_data_dict["Region"] = region_list

In [83]:
df_university_data = pd.DataFrame(uni_data_dict)
df_university_data["University"] = df_university_data["University"].apply(lambda x: x.replace("\n"," "))
df_university_data

,University,Location,Images,Region
0,Boston University,United States of America,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,America & Canada
1,Arizona State University,U. S. A.,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,America & Canada
2,The University of Pennsylvania,United States of America,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,America & Canada
3,Yale University,United States of America,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,America & Canada
4,Penn State University,United States of America,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,America & Canada
...,...,...,...,...
71,University of Sussex,United Kingdom,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,UK & Ireland
72,University of Warwick,United Kingdom,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,UK & Ireland
73,University of York,United Kingdom,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,UK & Ireland
74,University College London,"London, United Kingdom",https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,UK & Ireland


In [84]:
def set_country(country):
    if "U. S. A." in country or "USA" in country:
        return "United States of America"
    elif "Chile" in country:
        return "Chile"
    elif "Canada" in country:
        return "Canada"
    elif "Japan" in country:
        return "Japan"
    elif "South Korea" in country:
        return "South Korea"
    elif "Taiwan" in country:
        return "Taiwan"
    elif "Malaysia" in country:
        return "Malaysia"
    elif "Australia" in country or "Queensland" in country:
        return "Australia"
    elif "Netherlands" in country:
        return "Netherlands"
    elif "Germany" in country:
        return "Germany"
    elif "Croatia" in country:
        return "Croatia"
    elif "Russia" in country:
        return "Russia"
    elif "Lithuania" in country:
        return "Lithuania"
    elif "Spain" in country:
        return "Spain"
    elif "Italy" in country:
        return "Italy"
    elif "Czechia" in country:
        return "Czechia"
    elif "UK" in country or "United Kingdom" in country:
        return "United Kingdom"
    elif "Scotland" in country:
        return "Scotland"
    else:
        return country

In [85]:
df_university_data["Location"] = df_university_data["Location"].apply(set_country)

In [86]:
df_university_data.head(5)

,University,Location,Images,Region
0,Boston University,United States of America,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,America & Canada
1,Arizona State University,United States of America,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,America & Canada
2,The University of Pennsylvania,United States of America,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,America & Canada
3,Yale University,United States of America,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,America & Canada
4,Penn State University,United States of America,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,America & Canada


### Getting the Requirements and Country for each University

In [11]:
def get_digit(string):
    digit = ""
    for char in section:
        if char.isdigit() or char == ".":
            digit += char
    if digit == "":
        return None
    else:
        return float(digit)

def get_score(string):
    digit = ""
    for i in string:
        if i == " ":
            pass
        elif i.isdigit() or i == ".":
            digit += i
        else:
            break
    if digit == "":
        return None
    else:
        return float(digit)

def get_key(string):
    key = ""
    for i in string:
        if i.isalpha() or i == " ":
            key+= i
    return key

In [12]:
requirement_data = []
for region in dict_:
    for university in dict_[region]:
        data = {}
        driver = webdriver.Chrome()
        driver.get(university)
        try:
            driver.find_element(By.ID,"elementor-tab-title-4502").click()
            intake = driver.find_element(By.ID,'elementor-tab-content-4502').text
            
        except NoSuchElementException:
            driver.find_element(By.ID,"elementor-tab-title-1812").click()
            intake = driver.find_element(By.ID,'elementor-tab-content-1812').text
        intake = intake.split('\n')
        uni_name = driver.find_element(By.TAG_NAME,"h4").text
        data["Uni Name"] = uni_name
        for section in intake:
            if "GPA" in section:
                data["GPA"] = get_digit(section)
            elif "English Language" not in section:
                items = section.split(":")
                key = get_key(items[0])
                if "TOEFL" in key:
                    key = "TOEFL"
                elif "Duolingo" in key or "DET" in key:
                    key = "Duolingo English Test"
                try:
                    value = get_score(items[1])
                    data[key.strip()] = value
                except:
                    pass
        requirement_data.append(data)

In [13]:
#Creating the requirements dataframe
df_requirements = pd.DataFrame(requirement_data)
df_requirements.head(5)

,Uni Name,GPA,IELTS,TOEFL,Duolingo English Test,TOELF iBT
0,Boston University,3.0,6.5,84.0,110.0,NaN
1,Arizona State University,3.0,6.5,79.0,115.0,NaN
2,The University of Pennsylvania,3.3,7.0,100.0,120.0,NaN
3,Yale University,3.0,7.0,100.0,120.0,NaN
4,Penn State University,3.3,6.5,80.0,120.0,NaN


In [14]:
#Cleaning data because there is a typo in the scraped data
df_requirements['TOEFL'] = df_requirements['TOEFL'].fillna(df_requirements['TOELF iBT'])
df_requirements = df_requirements.drop("TOELF iBT",axis=1)

In [15]:
df_requirements.head(5)

,Uni Name,GPA,IELTS,TOEFL,Duolingo English Test
0,Boston University,3.0,6.5,84.0,110.0
1,Arizona State University,3.0,6.5,79.0,115.0
2,The University of Pennsylvania,3.3,7.0,100.0,120.0
3,Yale University,3.0,7.0,100.0,120.0
4,Penn State University,3.3,6.5,80.0,120.0


### Getting the Intake Data for each University

In [39]:
intake_data = {}
for region in dict_:
    for university in dict_[region]:
        driver = webdriver.Chrome()
        driver.get(university)
        
        try:
            driver.find_element(By.ID,"elementor-tab-title-4504").click()
            intake = driver.find_element(By.ID,'elementor-tab-content-4504').text
            
        except NoSuchElementException:
            driver.find_element(By.ID,"elementor-tab-title-1814").click()
            intake = driver.find_element(By.ID,'elementor-tab-content-1814').text
        
        intake = intake.split('\n')
        uni_name = driver.find_element(By.TAG_NAME,"h4").text
        title = ""
        items = {}
        data = {}
        
        for section in intake: #Ambil judul yang bold
            if ":" not in section:
                if (title != ""):
                    data[title] = items
                    
                title = section.strip()
                if title.count(" ") > 1:
                    front,mid,end = title.split(" ")
                    title = front + " " + end
                items = {}
                
            elif "Cofunding" in section: #Ambil judul buat cofunding
                data[title] = items
                title = title.split(" ")
                year = title[1]
                items = {}
                
                item = section.split(":")
                key = item[0].strip()
                values = item[1].strip()
                title = key + " " + year
                value = values.split(" ")
                student = int(value[0])
                items[key] = student
                
            else: #Datanya
                item = section.split(":")
                key = item[0].strip()
                    
                if "DET" in key or "Duolingo" in key:
                    key = "Duolingo English Test Score"
                
                if "TOEFL" in key:
                    key = "TOEFL Score"
            
                values = item[1].strip()
                if "students" in section.lower() or "student" in section.lower():
                    value = values.split(" ")
                    student = int(value[0])
                    items[key] = student
                else:
                    try:
                        value = values.split('-')
                        if (len(value) == 1):
                            value =values.split('–') 
                        min_score = float(value[0])
                        max_score = float(value[1])
                    except ValueError:
                        min_score = None
                        max_score = None
                    except IndexError:
                        min_score = values
                        max_score = values
                    items[key + " Min"] = min_score
                    items[key + " Max"] = max_score
            data[title] = items
        intake_data[uni_name] = data

In [40]:
#Combining "Regular Applicants", "Total Applicants", etc into "Total Applicants"
keys_to_remove = []
for key in intake_data.keys():
    for intake in intake_data[key].keys():
        for intake_key in intake_data[key][intake].keys():
            if "Applicants" in intake_key:
                keys_to_remove.append((key, intake, intake_key))

for key, intake, intake_key in keys_to_remove:
    intake_data[key][intake]["Total Students"] = intake_data[key][intake].pop(intake_key, None)

In [41]:
#Combining "Regular Awardees", "Total Awardees", etc into "Total Applicants"
keys_to_remove = []
for key in intake_data.keys():
    for intake in intake_data[key].keys():
        for intake_key in intake_data[key][intake].keys():
            if "Awardees" in intake_key:
                keys_to_remove.append((key, intake, intake_key))

for key, intake, intake_key in keys_to_remove:
    intake_data[key][intake]["Total Students"] = intake_data[key][intake].pop(intake_key, None)

In [42]:
#Creating a list for all the dictionary based on application and awardees
combined_intake_awardees = []
combined_intake_applicants = []
i = 0
for key in intake_data.keys():
    for intake in intake_data[key].keys():
        if "Awardees" in intake:
            intake_data[key][intake]["Uni Name"] = key
            intake_data[key][intake]["Intake"] = intake
            combined_intake_awardees.append(intake_data[key][intake])
        elif "Applicants" in intake:
            intake_data[key][intake]["Uni Name"] = key
            intake_data[key][intake]["Intake"] = intake
            combined_intake_applicants.append(intake_data[key][intake])

In [43]:
#Creating awardees and applicants dataframe
df_awardees = pd.DataFrame(combined_intake_awardees)
df_applicants = pd.DataFrame(combined_intake_applicants)

In [44]:
df_applicants.head(5)

,GPA Min,GPA Max,TOEFL Score Min,TOEFL Score Max,IELTS Score Min,IELTS Score Max,Duolingo English Test Score Min,Duolingo English Test Score Max,Total Students,Uni Name,Intake
0,3.12,4.00,88.0,106.0,6.5,8.5,0.0,155.0,395,Boston University,Applicants 2022
1,2.89,3.99,67.0,67.0,5.5,8.5,80.0,155.0,231,Boston University,Applicants 2023
2,3.31,3.81,None,None,6.5,6.5,100.0,140.0,9,Boston University,Cofunding Applicants 2023
3,3.01,3.99,64.0,97.0,6.5,8.5,100.0,150.0,190,Arizona State University,Applicants 2022
4,3.16,3.99,None,None,6.5,8.0,90.0,150.0,68,Arizona State University,Applicants 2023


In [45]:
df_awardees.head(5)

,GPA Min,GPA Max,TOEFL Score Min,TOEFL Score Max,IELTS Score Min,IELTS Score Max,Duolingo English Test Score Min,Duolingo English Test Score Max,Total Students,Uni Name,Intake
0,3.39,3.96,None,None,8.5,8.5,140.0,155.0,20,Boston University,Awardees 2022
1,3.40,3.90,None,None,8.5,8.5,140.0,155.0,20,Boston University,Awardees 2023
2,3.44,3.94,None,None,None,None,140.0,145.0,8,Arizona State University,Awardees 2022
3,3.16,3.98,None,None,7.5,7.5,130.0,145.0,11,Arizona State University,Awardees 2023
4,3.47,4.00,None,None,8.5,8.5,145.0,160.0,20,The University of Pennsylvania,Awardees 2022


In [46]:
#Cleaning a typo from the website as there is there is a "Total Applicants" field in the awardee section
# df_awardees['Total Students'] = df_awardees['Total Students'].fillna(df_awardees['Total Applicants'])
# df_awardees = df_awardees.drop("Total Applicants",axis=1)
df_awardees["Total Students"] = df_awardees["Total Students"].apply(lambda x:int(x))

In [47]:
df_combined = pd.concat([df_awardees, df_applicants])

In [48]:
df_combined

,GPA Min,GPA Max,TOEFL Score Min,TOEFL Score Max,IELTS Score Min,IELTS Score Max,Duolingo English Test Score Min,Duolingo English Test Score Max,Total Students,Uni Name,Intake
0,3.39,3.96,None,None,8.5,8.5,140.0,155.0,20,Boston University,Awardees 2022
1,3.40,3.90,None,None,8.5,8.5,140.0,155.0,20,Boston University,Awardees 2023
2,3.44,3.94,None,None,None,None,140.0,145.0,8,Arizona State University,Awardees 2022
3,3.16,3.98,None,None,7.5,7.5,130.0,145.0,11,Arizona State University,Awardees 2023
4,3.47,4.00,None,None,8.5,8.5,145.0,160.0,20,The University of Pennsylvania,Awardees 2022
...,...,...,...,...,...,...,...,...,...,...,...
140,3.00,3.96,114,114,6.5,8.0,101.0,112.0,37,University of York,Applicants 2023
141,3.28,4.00,110.0,111.0,7.5,8.5,105.0,135.0,24,University College London,Applicants 2022
142,3.29,3.97,108,108,7.0,8.5,None,None,65,University College London,Applicants 2023
143,3.04,4.00,103.0,110.0,6.0,8.0,100,100,26,Queen Mary University of London,Applicants 2022


In [52]:
df_combined[['Type', 'Year']] = df_combined['Intake'].str.extract(r'([A-Za-z\s]+)\s(\d+)')

In [55]:
df_combined = df_combined.drop("Intake",axis=1)

array(['Awardees', 'Cofunding Awardees', 'Applicants',
       'Cofunding Applicants'], dtype=object)

In [90]:
df_combined["Uni Name"] = df_combined["Uni Name"].apply(lambda x: x.replace("\n"," "))
df_requirements["Uni Name"] = df_requirements["Uni Name"].apply(lambda x: x.replace("\n"," "))

In [57]:
df_combined

,GPA Min,GPA Max,TOEFL Score Min,TOEFL Score Max,IELTS Score Min,IELTS Score Max,Duolingo English Test Score Min,Duolingo English Test Score Max,Total Students,Uni Name,Type,Year
0,3.39,3.96,None,None,8.5,8.5,140.0,155.0,20,Boston University,Awardees,2022
1,3.40,3.90,None,None,8.5,8.5,140.0,155.0,20,Boston University,Awardees,2023
2,3.44,3.94,None,None,None,None,140.0,145.0,8,Arizona State University,Awardees,2022
3,3.16,3.98,None,None,7.5,7.5,130.0,145.0,11,Arizona State University,Awardees,2023
4,3.47,4.00,None,None,8.5,8.5,145.0,160.0,20,The University of Pennsylvania,Awardees,2022
...,...,...,...,...,...,...,...,...,...,...,...,...
140,3.00,3.96,114,114,6.5,8.0,101.0,112.0,37,University of York,Applicants,2023
141,3.28,4.00,110.0,111.0,7.5,8.5,105.0,135.0,24,University College London,Applicants,2022
142,3.29,3.97,108,108,7.0,8.5,None,None,65,University College London,Applicants,2023
143,3.04,4.00,103.0,110.0,6.0,8.0,100,100,26,Queen Mary University of London,Applicants,2022


## Changing NaN to None

In [61]:
df_combined.replace({np.nan: None}, inplace=True)
df_requirements.replace({np.nan: None}, inplace=True)

## Dumping to JSON File

In [91]:
file_path_list = []
#Changing the path for each table.json
file_path_list += ["iisma_intake.json"]
file_path_list += ["iisma_university.json"] 
file_path_list += ["iisma_requirements.json"]

In [107]:
json_list = []

json_list += [json.dumps(df_combined.to_dict(orient='records'), indent=2)]

json_list += [json.dumps(df_university_data.to_dict(orient='records'), indent=2)]

json_list += [json.dumps(df_requirements.to_dict(orient='records'), indent=2)]

j = 0
for i in file_path_list:
    with open(i, "w") as json_file:
        json_file.write(json_list[j])
    j+=1

In [64]:
df_combined[df_combined["Type"] == "Cofunding Applicants"]

,GPA Min,GPA Max,TOEFL Score Min,TOEFL Score Max,IELTS Score Min,IELTS Score Max,Duolingo English Test Score Min,Duolingo English Test Score Max,Total Students,Uni Name,Type,Year
2,3.31,3.81,None,None,6.5,6.5,100.0,140.0,9,Boston University,Cofunding Applicants,2023
16,3.15,3.91,45.3,45.3,6.5,7.5,110.0,145.0,41,Michigan State University,Cofunding Applicants,2023
30,2.95,4.0,94.0,94.0,6.0,7.5,70.0,150.0,84,Hanyang University (Seoul Campus),Cofunding Applicants,2023
37,2.95,3.98,None,None,6.0,7.5,75.0,150.0,71,Universiti Malaya,Cofunding Applicants,2023
40,3.31,4.0,None,None,6.0,8.0,95.0,150.0,52,Universiti Kebangsaan Malaysia,Cofunding Applicants,2023
50,3.51,3.67,None,None,None,None,120.0,130.0,3,Chulalongkorn University,Cofunding Applicants,2023
53,3.21,4.0,None,None,6.0,7.5,70.0,140.0,38,Middle East Technical University,Cofunding Applicants,2023
90,3.39,3.91,None,None,None,None,105.0,140.0,15,Humboldt-Universität zu Berlin,Cofunding Applicants,2023
114,3.03,4.0,95,95,6.5,8.5,None,None,35,Lancaster University,Cofunding Applicants,2023


In [89]:
df_university_data[df_university_data["Region"] == "Asia"]

,University,Location,Images,Region
14,Osaka University,Japan,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,Asia
15,Singapore Management University,Singapore,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,Asia
16,Hanyang University (Seoul Campus),South Korea,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,Asia
17,National Taiwan University of Science and Tech...,Taiwan,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,Asia
18,Korea University,South Korea,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,Asia
19,Universiti Malaya,Malaysia,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,Asia
20,Universiti Kebangsaan Malaysia,Malaysia,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,Asia
21,Universiti Sains Malaysia,Malaysia,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,Asia
22,Nanyang Technological University,Singapore,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,Asia
23,Keio University,Japan,https://i0.wp.com/iisma.kemdikbud.go.id/info/w...,Asia


In [94]:
df_university_data.to_excel("University.xlsx")
df_combined.to_excel("intake.xlsx")
df_requirements.to_excel("requirements.xlsx")

In [106]:
df_requirements.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

In [99]:
df_university_data.columns

Index(['University', 'Location', 'Images', 'Region'], dtype='object')